<a href="https://colab.research.google.com/github/AnaOliari/API-prova/blob/main/DataScience.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files


uploaded = files.upload()

In [ ]:
import pandas as pd
import pandas as pd
import io

df = pd.read_excel(io.BytesIO(uploaded['Lista NPS Positivo_V4.xlsx']))
print(df)

QUESTÃO 1

1- Utilizando a base inteira, calcule o target principal, formando uma nova coluna na base, onde a variável nota será analisada em cada linha: a. Notas 9 ou 10 : a coluna target irá conter a classe “promotor”. No total a coluna deve conter 18.251 promotores, sendo essa a classe majoritária. b. Notas 8 ou 7: a coluna target irá conter a classe “neutro”. No total a coluna deve conter 4.738 neutros. c. Notas < 7: a coluna target irá conter a classe “detrator”. No total a coluna deve conter 2.185 detratores, sendo essa a classe minoritária. d. Executar o comando df.groupby(“target”).count() e verificar se as contagens do target batem com a descrição dos itens a, b e c.

In [ ]:
import pandas as pd
#Função para calcular o target, separando o que é promotor, neutro e detrator
def calcular_target(nota):
  if nota in (9, 10):
    return "promotor"
  elif nota in (7, 8):
    return "neutro"
  elif nota < 7:
    return "detrator"
  else:
    return None  # Trata casos de notas inválidas, se houver

# Aplicando a função para criar a coluna 'target'
df['target'] = df['nota'].apply(calcular_target)

In [ ]:
contagem_Classes = df.groupby('target').count()
print(contagem_Classes)

In [ ]:
df.groupby("target").count()

QUESTÃO 2

2- Localizar a variável “mercado” e filtrar para reter apenas instâncias que são do brasil, para isso você pode utilizar um comando como:
df = df.loc[df[”mercado”] ==”brasil”].

In [ ]:
# Verificar a presença da coluna 'mercado' e filtrar os registros para 'BRASIL'
if 'mercado' in df.columns:
    df_brasil = df[df['mercado'].str.upper() == 'BRASIL']
    total_brasil = len(df_brasil)
else:
    total_brasil = "A coluna 'mercado' não está presente no DataFrame."

total_brasil

QUESTÃO 3

3- Assim como no item 2, executar um comando de filtragem para trabalhar com o grupo que a sua equipe ficou encarregada. O nome da variável a ser filtrada é “Grupo de Produto”, exemplo : df = df.loc[df[”Grupo de Produto”] == “Grupo x”], onde x é o número de seu grupo.

In [ ]:
# Verificar a presença da coluna 'Grupo de Produto' e filtrar para o Grupo 4
if 'Grupo de Produto' in df_brasil.columns:
    grupo4_df = df_brasil[df_brasil['Grupo de Produto'] == 'Grupo 4']
    total_grupo4 = len(grupo4_df)
else:
    total_grupo4 = "A coluna 'Grupo de Produto' não está presente no DataFrame."

total_grupo4

QUESTÃO 4

4- Fazer a volumetria de target, calculando para o seu grupo quantos promotores, neutros e detratores ficaram na base, calcular também o percentual de cada classe.

In [ ]:
# Calcular a volumetria e o percentual dos targets no Grupo 4
volumetria_grupo4 = grupo4_df['target'].value_counts()
percentual_grupo4 = (volumetria_grupo4 / volumetria_grupo4.sum()) * 100

volumetria_grupo4, percentual_grupo4

QUESTÃO 5

5- Criar uma coluna chamada região, que irá corresponder as 5 regiões do país, baseado na informação da coluna “estado”. Ex: se o estado for Paraná, Santa Catarina ou Rio grande do Sul, a coluna região deve conter a string “sul”.

In [ ]:
# Mapeamento das regiões brasileiras
regioes = {
    'Norte': ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
    'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE'],
    'Centro-Oeste': ['DF', 'GO', 'MT', 'MS'],
    'Sudeste': ['ES', 'MG', 'RJ', 'SP'],
    'Sul': ['PR', 'RS', 'SC']
}

# Função para atribuir a região com base no estado
def atribuir_regiao(estado):
    for regiao, estados in regioes.items():
        if estado in estados:
            return regiao
    return None  # Caso o estado não seja encontrado no mapeamento

# Criar a coluna 'região' no DataFrame do Grupo 4
if 'estado' in grupo4_df.columns:
    grupo4_df['região'] = grupo4_df['estado'].apply(atribuir_regiao)
    regioes_distribuicao = grupo4_df['região'].value_counts()
else:
    regioes_distribuicao = "A coluna 'estado' não está presente no DataFrame."

regioes_distribuicao

QUESTÃO 6

6- Criar uma coluna chamada safra, baseada na variável “data_resposta”, capturando apenas o ano da resposta.

In [ ]:
# Converter 'data_resposta' para datetime e criar a coluna 'safra'
if 'data_resposta' in grupo4_df.columns:
    grupo4_df['data_resposta'] = pd.to_datetime(grupo4_df['data_resposta'], errors='coerce')
    grupo4_df['safra'] = grupo4_df['data_resposta'].dt.year
    safra_distribuicao = grupo4_df['safra'].value_counts()
else:
    safra_distribuicao = "A coluna 'data_resposta' não está presente no DataFrame."

safra_distribuicao

QUESTÃO 7 PARTE 1

7- Calcular a volumetria do target Safrada (pelo ano) para a base total (sempre filtrada para o seu grupo), fazer a mesma volumetria para cada região do país e para cada um dos quatro períodos de pesquisa baseado na coluna “Período de Pesquisa”. A volumetria deve ser calculada em valores absolutos e percentuais. Além do cálculo safrado, a volumetria deve somar todas as safras no final. Fazer verificações se o total é coerente com a contagem obtida diretamente da base.

In [ ]:
# Função para calcular a volumetria por safra
def calcular_volumetria_safra(dataframe, coluna_safra='safra', coluna_target='target'):
    safra_volumetria = dataframe.groupby(coluna_safra)[coluna_target].value_counts().unstack(fill_value=0)
    safra_volumetria['Total'] = safra_volumetria.sum(axis=1)
    safra_volumetria['%Promotores'] = (safra_volumetria.get('promotor', 0) / safra_volumetria['Total']) * 100
    safra_volumetria['%Neutros'] = (safra_volumetria.get('neutro', 0) / safra_volumetria['Total']) * 100
    safra_volumetria['%Detratores'] = (safra_volumetria.get('detrator', 0) / safra_volumetria['Total']) * 100
    safra_volumetria.loc['Total'] = safra_volumetria.sum()  # Somar todas as safras
    safra_volumetria.loc['Total', '%Promotores'] = (safra_volumetria.loc['Total', 'promotor'] / safra_volumetria.loc['Total', 'Total']) * 100
    safra_volumetria.loc['Total', '%Neutros'] = (safra_volumetria.loc['Total', 'neutro'] / safra_volumetria.loc['Total', 'Total']) * 100
    safra_volumetria.loc['Total', '%Detratores'] = (safra_volumetria.loc['Total', 'detrator'] / safra_volumetria.loc['Total', 'Total']) * 100
    return safra_volumetria

# Calcular a volumetria para a base total por safra
volumetria_total_safra = calcular_volumetria_safra(grupo4_df)

# Exibir a tabela de volumetria diretamente no ambiente Google Colab
volumetria_total_safra

QUESTÃO 7 PARTE 2

7 Após o cálculo das volumetrias, inferir se existe diferença de balanceamento entre as classes, fazer isso para as safras e para o total, indicando no relatório a classe majoritária e a classe minoritária. Também indicar no relatório se alguma safra apresenta uma volumetria muito diferente comparada com as demais safras. Repetir essa análise fazendo novas tabelas de volumetria para cada filtragem do seu grupo, sendo que no total deve ter: (i) uma tabela de volumetria para Base total, (ii) 5 tabelas de volumetrias para as regiões e (iv) 4 tabelas de volumetrias para os períodos de pesquisas. No total, o relatório deverá conter 10 tabelas de volumetria igual a este exemplo com suas respectivas análises.

In [ ]:
# Ajustar a função para lidar com categorias ausentes
def calcular_volumetria_safra_ajustada(dataframe, coluna_safra='safra', coluna_target='target'):
    safra_volumetria = dataframe.groupby(coluna_safra)[coluna_target].value_counts().unstack(fill_value=0)
    # Garantir que todas as categorias estão presentes
    for categoria in ['promotor', 'neutro', 'detrator']:
        if categoria not in safra_volumetria.columns:
            safra_volumetria[categoria] = 0

    safra_volumetria['Total'] = safra_volumetria.sum(axis=1)
    safra_volumetria['%Promotores'] = (safra_volumetria['promotor'] / safra_volumetria['Total']) * 100
    safra_volumetria['%Neutros'] = (safra_volumetria['neutro'] / safra_volumetria['Total']) * 100
    safra_volumetria['%Detratores'] = (safra_volumetria['detrator'] / safra_volumetria['Total']) * 100

    safra_volumetria.loc['Total'] = safra_volumetria.sum()
    safra_volumetria.loc['Total', '%Promotores'] = (safra_volumetria.loc['Total', 'promotor'] / safra_volumetria.loc['Total', 'Total']) * 100
    safra_volumetria.loc['Total', '%Neutros'] = (safra_volumetria.loc['Total', 'neutro'] / safra_volumetria.loc['Total', 'Total']) * 100
    safra_volumetria.loc['Total', '%Detratores'] = (safra_volumetria.loc['Total', 'detrator'] / safra_volumetria.loc['Total', 'Total']) * 100
    return safra_volumetria

# Recalcular volumetria por região com ajuste
def recalcular_volumetria_por_regiao(dataframe, coluna_regiao='região', coluna_safra='safra', coluna_target='target'):
    regioes = dataframe[coluna_regiao].unique()
    regioes_volumetria = {}
    for regiao in regioes:
        if pd.notna(regiao):  # Ignorar valores nulos
            df_regiao = dataframe[dataframe[coluna_regiao] == regiao]
            regioes_volumetria[regiao] = calcular_volumetria_safra_ajustada(df_regiao, coluna_safra, coluna_target)
    return regioes_volumetria

# Recalcular
volumetria_por_regiao_corrigida = recalcular_volumetria_por_regiao(grupo4_df)

# Exibir as tabelas de volumetria por região diretamente no console
for regiao, tabela in volumetria_por_regiao_corrigida.items():
    print(f"\nVolumetria Ajustada para a Região {regiao}:\n")
    print(tabela)

QUESTÃO 8

8- Filtrar as perguntas pertencentes ao seu grupo. Descartar quaisquer colunas que não sejam necessárias para a continuidade da análise.

QUESTÃO 9

9- Em um primeiro momento fazer para o seu grupo a correlação de spearman, entre a variável nota e as demais variáveis de perguntas (contendo a nota de 0 a 10). Ordenar a lista de correlações da maior correlação para a menor, grifando em vermelho as correlações fortes, grifando em azul as correlações médias e grifando em verde as correlações fracas. Fazer isso para o seu grupo inteiro, por região e por período de pesquisa. Apresentar também a lista de correlações Safrada, uma lista por safra, replicando o trabalho para cada região e período de pesquisa.

In [ ]:
# Filtrar o Grupo 4
grupo4_df = df[df['Grupo de Produto'] == 'Grupo 4']

# Selecionar colunas relevantes para análise de correlação
colunas_escaladas = [col for col in grupo4_df.columns if grupo4_df[col].dtype in ['float64', 'int64'] and col != 'nota']

# Preparar DataFrame para correlação
grupo4_corr_df = grupo4_df[['nota'] + colunas_escaladas]

# Calcular a correlação de Spearman
spearman_corr = grupo4_corr_df.corr(method='spearman')

# Ordenar as correlações da maior para a menor em relação à 'nota'
spearman_corr_sorted = spearman_corr['nota'].drop('nota').sort_values(ascending=False)

# Destacar as correlações com cores conforme os critérios
def destacar_correlacao(valor):
    if abs(valor) >= 0.7:
        return f"🔴 {valor:.3f}"  # Vermelho (forte)
    elif 0.4 <= abs(valor) < 0.7:
        return f"🔵 {valor:.3f}"  # Azul (média)
    else:
        return f"🟢 {valor:.3f}"  # Verde (fraca)

# Aplicar destaque
spearman_corr_destacada = spearman_corr_sorted.apply(destacar_correlacao)

# Exibir as correlações destacadas
spearman_corr_destacada


TESTE DO MODELO PARA VER PELA CURVA SE O MODELO ESTA BOM OU RUIM

In [ ]:
# Recriar as colunas necessárias para os targets binários
grupo4_df['target'] = grupo4_df['nota'].apply(calcular_target)

# Criar os targets binários novamente
grupo4_df['target_detrator'] = (grupo4_df['target'] == 'detrator').astype(int)
grupo4_df['target_neutro'] = (grupo4_df['target'] == 'neutro').astype(int)

# Confirmar a criação das colunas
grupo4_df[['target', 'target_detrator', 'target_neutro']].head()


In [ ]:
# Repetir o processo de treinamento com os dados corrigidos
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score


# Assuming 'colunas_escaladas' from your previous code contains the features
X_cleaned = grupo4_df[colunas_escaladas]  # Define X_cleaned using your feature columns

# Dados para o modelo de Detratores
y_detrator = grupo4_df['target_detrator']
X_train_detrator, X_test_detrator, y_train_detrator, y_test_detrator = train_test_split(X_cleaned, y_detrator, test_size=0.2, random_state=42)

# Dados para o modelo de Neutros
y_neutro = grupo4_df['target_neutro']
X_train_neutro, X_test_neutro, y_train_neutro, y_test_neutro = train_test_split(X_cleaned, y_neutro, test_size=0.2, random_state=42)

# Treinar os modelos
rf_detrator = RandomForestClassifier(random_state=42)
rf_detrator.fit(X_train_detrator, y_train_detrator)

rf_neutro = RandomForestClassifier(random_state=42)
rf_neutro.fit(X_train_neutro, y_train_neutro)

# Fazer previsões
y_pred_detrator = rf_detrator.predict(X_test_detrator)
y_pred_neutro = rf_neutro.predict(X_test_neutro)

# Avaliar os modelos
report_detrator = classification_report(y_test_detrator, y_pred_detrator, output_dict=True)
report_neutro = classification_report(y_test_neutro, y_pred_neutro, output_dict=True)

# AUC-ROC para ambos
roc_auc_detrator = roc_auc_score(y_test_detrator, rf_detrator.predict_proba(X_test_detrator)[:, 1])
roc_auc_neutro = roc_auc_score(y_test_neutro, rf_neutro.predict_proba(X_test_neutro)[:, 1])

# Consolidar resultados
results_final_cleaned_models = {
    "Modelo Detrator": {
        "AUC-ROC": roc_auc_detrator,
        "Relatório de Classificação": report_detrator,
    },
    "Modelo Neutro": {
        "AUC-ROC": roc_auc_neutro,
        "Relatório de Classificação": report_neutro,
    },
}

results_final_cleaned_models

QUESTÃO 10

10- Faça 2 modelos de classificação binária por análise (um para neutro e outro para detrator). Para isso, use apenas as variáveis numéricas (perguntas) como variáveis de entrada (X) e treine o modelo com um novo target, reduzindo o target de 3 classes para 2 classes, transformando o problema multi-classe e um problema de classificação binária (exemplo 1: modelo de detratores, positivar caso detrator e negativar caso neutro ou promotor) (y) (exemplo 2: modelo de neutros, positivar caso neutro e negativar caso detrator ou promotor) (y). Importante!! Ao criar o target binário, não utilize no espaço de características do modelo (X) o target de 3 classes como entrada, nem a variável nota, pois ambas são consideradas vazamento neste contexto (pois derivam o target). Sendo assim, para cada modelo, o X (espaço de características) deve conter todas as colunas de perguntas, menos a variável nota e o y (target) deve conter apenas o target binário adaptado.

QUESTÃO 11

11- No total devem ser feitos 20 modelos: um modelo detrator e um modelo neutro para a base inteira filtrada pelo seu grupo, repetindo a análise por: região e período de pesquisa. Não precisa fazer modelos safrados!

QUESTÃO 12

12- Colocar no relatório as top 10 variáveis de cada modelo, para isso use um modelo como RandomForest ou XGbooost. Tirar as conclusões sobre top variáveis, podendo comparar esta análise com a lista de correlações.

QUESTÃO 13

13- Tentar avaliar as 5 top-variáveis de cada modelo com uma das técnica de gráfico, podendo ser PDP, ALE ou SHAP. A explicação de como fazer estes gráficos será fornecida nas próximas aulas, antes da avaliação.